In [50]:
# The core class, everything in the card game revolves around cards
class Card:
    def __init__(self, cardid, name, clan, nation, grade, power, shield, critical = 1,
    skill1 = None, skill2 = None, skill3 = None, marker = None, istrigger = False,
    triggertype = None, boost = False, intercept = False, drive = 1, flavor = None,
    image = None, tags = []):
        self.cardid = cardid
        self.name = name
        self.clan = clan
        self.nation = nation
        self.grade = grade
        self.power = power
        self.shield = shield
        self.critical = critical
        self.skill1 = skill1
        self.skill2 = skill2
        self.skill3 = skill3
        self.marker = marker
        self.istrigger = istrigger
        self.triggertype = triggertype
        self.boost = boost
        self.intercept = intercept
        self.drive = drive
        self.flavor = flavor
        self.image = image
        self.tags = tags
    
    def get_name(self):
        return self.name
    
    def get_id(self):
        return self.cardid
    
### Example Little_Sage_Marron = Card(1,"Little Sage Marron","Royal Paladin","United Sanctuary",1,8000,10000, boost = True)

# Define Game Cards, Cards that will be used in the game

class Gamecard(Card):
    def __init__(self
                 , *args):
        self.__dict__ = args[0].__dict__.copy()
        self.faceup = True
        self.boostedpower = 0
        self.currentcritical = self.critical
        self.currentdrive = self.drive
        self.equipgauge = []
        self.isrest = False
        self.currentshield = self.shield
        self.canintercept = self.intercept
        self.canboost = self.boost
    
    def counter_blast(self):
        if self.faceup == True:
            self.faceup = False
        else:
            print('Error - Already Facedown')
    
    def counter_charge(self):
        if self.faceup == False:
            self.faceup = True
        else:
            print('Error - Already Faceup')

    def current_power(self):
        return self.boostedpower + self.power
        

# Define Circles


class Circle:
    def __init__(self, name, row, column, card = None, isaccel = False, marker = None, markercount = 0, isvanguard = False):
        self.name = name
        self.row = row
        self.column = column
        self.isaccel = isaccel
        self.marker = marker
        self.card = card
        self.markercount = markercount
        self.isvanguard = isvanguard

    def add_marker(self,marker):
        self.marker = marker
        self.markercount += 1
    def retire(self):
        if self.card != None:
            dropzone.add_card(self.card)
            if len(self.card.equipgauge) > 0:
                for i in equipgauge:
                    dropzone.add_card(i)
        self.card = None
    def call_card(self,card):
        self.retire()
        self.card = card
        


# Define Zones


class Zone:
    def __init__(self, name, cardlist = None):
        self.name = name
        if cardlist is None:
            cardlist = []
        self.cardlist = cardlist

    def add_card(self,card,bottom=False):
        if not bottom:
            self.cardlist.append(card)
        else:
            self.cardlist.insert(0,card)
        
    def remove_card(self,card):
        self.cardlist.remove(card)
    
    def empty_list(self):
        self.cardlist = []
        


# Define Players


class Player:
    def __init__(self, name, decklist, field = [],isactive = False, firstvan = None, clan = None, nation = None):
        self.name = name
        self.decklist = decklist
        self.clan = clan
        self.nation = nation
        self.field = field
        self.isactive = isactive
        self.firstvan = firstvan
        self.rightfront = Circle('rightfront',1,1)
        self.leftfront = Circle('leftfront',1,3)
        self.centerfront = Circle('centerfront',1,2,isvanguard = True)
        self.rightback = Circle('rightback',2,1)
        self.leftback = Circle('leftback',2,3)
        self.centerback = Circle('centerback',2,2)
        self.field = [self.leftfront, self.centerfront, self.rightfront,
                      self.leftback, self.centerback, self.rightback]
        self.deckzone = Zone('Deck')
        self.dropzone = Zone('Drop')
        self.guardzone = Zone('Guard')
        self.triggerzone = Zone('Trigger')
        self.bindzone = Zone('Bind')
        self.damagezone = Zone('Damage')
        self.handzone = Zone('Hand')
        self.soulzone = Zone('Soul')
        self.gzone = Zone('G')
        self.assistzone = Zone('Assist')
        
        
    def deck_init(self):
        self.deckzone.cardlist = []
        for i in self.decklist:
            if self.decklist[i] == -1:
                self.firstvan = Gamecard(carddb[i])
            else:
                for n in range(self.decklist[i]):
                    self.deckzone.cardlist.append(Gamecard(carddb[i]))
        shuffle(self.deckzone.cardlist)
        
        # ex. p1dek = {5: -1, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 2, 14: 4, 15: 4, 16: 3, 17: 4, 18: 4}
    
    def set_firstvan(self):
        self.centerfront.call_card(self.firstvan)
        
    def ride_card(self,ride):
        # attempt When this card is ridden upon skills
        self.soulzone.add_card(self.centerfront.card)
        self.centerfront.card = ride
        # attempt When Placed on VC skills
        
        
class Skill:
    def __init__(self, skillid, name, kind, effect, requirement, area, timing = []):
        self.skillid = skillid
        self.name = name
        self.kind = kind
        self.effect = effect
        self.requirement = requirement #ex. If cost is Soul Blast 2, check soul has >= 2 cards.
        self.area = area
        self.timing = timing
        
    def get_timings(self):
        return self.timing
    
    def use_skill(self):
        return skillid #here i'd want it to call the effect of that skill... somehow
    
    def is_continuous(self):
        return self.kind == 'Cont'

In [63]:
import random as rm 

def decide_first(you,opp):
    global player1
    global player2
    if rm.randint(0,1) == 0:
        player1 = you
        player2 = opp
        print('You are going First!')
    else:
        player1 = opp
        player2 = you
        print('You are going Second!')

def change_turnplayer():
    global player1
    global player2
    global turnplayer
    if turnplayer == player1:
        turnplayer = player2
    elif turnplayer == player2:
        turnplayer = player1

def get_opponent(player):
    if player == player1:
        return player2
    if player == player2:
        return player1
        
def shuffle(deck):
    rm.shuffle(deck)

def select_from(z):
    #Takes list, allows player to select from list of cards, circles, etc
    print('Select from the following:')
    for i in range(len(z)):
        print(str(i) + ' : ' + z[i].name)
    while True:
        selection = int(input())
        if selection >= 0 and selection <= len(z):
            return z[selection]
        else:
            print('Error - not in list')

def select_from_list(z):
    #Takes list, allows player to select from list of cards, circles, etc
    print('Select from the following:')
    for i in range(len(z)):
        print(str(i) + ' : ' + z[i])
    while True:
        selection = int(input())
        if selection >= 0 and selection <= len(z):
            return z[selection]
        else:
            print('Error - not in list')
            
def draw(player, num):
    for n in range(num):
        if len(player.deckzone.cardlist) >= num:
            drawn = player.deckzone.cardlist.pop()
            player.handzone.add_card(drawn)
        else:
            print('No more cards in deck - Game Over')
            # gameover procedure here

def mulligan(player):
    print('Your hand is currently:')
    print([i.name for i in player.handzone.cardlist])
    print('How many cards would you like to mulligan?')
    numback = 0
    while numback <= 5 and numback >= 0:
        numback = int(input())
        for n in range(numback):
            print('Please choose a card:')
            chosen = select_from(player.handzone.cardlist)
            player.handzone.remove_card(chosen)
            player.deckzone.add_card(chosen,bottom=True)
        draw(player, numback)
        shuffle(player.deckzone.cardlist)
        numback = -1
    
            
def drive_check(player,num,ezel = False):
    if ezel == False:
        for n in range(num):
            if len(player.deckzone.cardlist) >= num:
                drawn = player.deckzone.cardlist.pop()
                player.triggerzone.add_card(drawn)
                print('Drive Check - {}'.format(drawn.name))
                if drawn.istrigger == True:
                    print('Get! {} Trigger! Power + 10000!'.format(drawn.triggertype))
                    if drawn.triggertype == 'Front':
                        front_trigger(player)
                    else:
                        print('Please Select a Circle with a Unit')
                        fieldlist = list(filter(lambda x: (x.card != None), player.field))
                        powunit = select_from(fieldlist)
                        powunit.card.boostedpower += 10000
                        if drawn.triggertype == 'Critical':
                            critical_trigger(player)
                        elif drawn.triggertype == 'Heal':
                            heal_trigger(player,get_opponent(player))                     
                        elif drawn.triggertype == 'Draw':
                            draw_trigger(player)
                        elif drawn.triggertype == 'Stand':
                            stand_trigger(player)
                trigcard = player.triggerzone.cardlist.pop()
                player.handzone.add_card(trigcard)
            else:
                print('No more cards in deck - Game Over')
                #gameover procedure here
    else:
        pass
        #add in Blazing Lion, Platina Ezel skill effect

def damage_check(player,num):
    for n in range(num):
        if len(player.deckzone.cardlist) >= num:
            drawn = player.deckzone.cardlist.pop()
            player.triggerzone.add_card(drawn)
            print('Drive Check - {}'.format(drawn.name))
            if drawn.istrigger == True:
                print('Get! {} Trigger! Power + 10000!'.format(drawn.triggertype))
                if drawn.triggertype == 'Front':
                    front_trigger(player)
                else:
                    print('Please Select a Circle with a Unit')
                    fieldlist = list(filter(lambda x: (x.card != None), player.field))
                    powunit = select_from(fieldlist)
                    powunit.card.boostedpower += 10000
                    if drawn.triggertype == 'Critical':
                        critical_trigger(player)
                    elif drawn.triggertype == 'Heal':
                        heal_trigger(player,get_opponent(player))                     
                    elif drawn.triggertype == 'Draw':
                        draw_trigger(player)
                    elif drawn.triggertype == 'Stand':
                        stand_trigger(player)
            trigcard = player.triggerzone.cardlist.pop()
            player.damagezone.add_card(trigcard)
        else:
            print('No more cards in deck - Game Over')
    if len(player.deckzone.cardlist) == 0 or len(player.damagezone.cardlist) == 6:
        print('Game Over')
        #Game End Procedure would be kicked off here
        
def discard(card):
    if card in player.handzone.cardlist:
        player.handzone.remove_card(card)
        player.dropzone.add_card(card)
    else:
        print('Error - Card not in Hand...')

def guard_power(zone):
    shieldsum = 0
    for i in zone.cardlist:
        shieldsum += i.currentshield
    return shieldsum

### Triggers ###

def front_trigger(player):
    for circ in player.field:
        if circ.row == 1 and circ.card:
            circ.card.boostedpower += 10000

def draw_trigger(player):
    draw(player,1)

def critical_trigger(player):
    critfield = list(filter(lambda x: (x.card != None), player.field))
    critunit = select_from(critfield)
    critunit.card.currentcritical += 1

def heal_trigger(player,opponent):
    if len(player.damagezone.cardlist) >= len(opponent.damagezone.cardlist):
        #select card in damagezone
        healcard = select_from(player.damagezone.cardlist)
        player.damagezone.remove_card(healcard)
        player.dropzone.add_card(healcard)
    else:
        pass

def stand_trigger(player):
    standfield = list(filter(lambda x: (x.card != None), player.field))
    standunit = select_from(standfield.remove('centerfront'))
    standunit.card.isrest = False
            
### end of Triggers ###

def sentinel():
    global sentinel
    sentinel = True

def end_of_battle(player):
    sentinel = False
    if len(player.guardzone.cardlist) > 0:
        for i in player.guardzone.cardlist:
            player.guardzone.remove_card(i)
            player.dropzone.add_card(i)

def add_equip_guage(player,card,num):
    for n in range(num):
        if len(player.deckzone.cardlist) >= num:
            topcard = player.deckzone.cardlist.pop()
            card.equipgauge.append(topcard)
        else:
            print('No more cards in deck - Game Over')

def soul_charge(player,num):
    for n in range(num):
        if len(player.deckzone.cardlist) >= num:
            charged = player.deckzone.cardlist.pop()
            player.soulzone.add_card(charged)
        else:
            print('No more cards in deck - Game Over')

def soul_blast(player,num):
    for n in range(num):
        print('Soulblast:')
        soulcard = select_from(soulzone)
        soulzone.remove_card(soulcard)
        dropzone.add_card(soulcard)

def g_assist(player):
    currentgrade = player.centerfront.card.grade
    checked = player.deckzone.cardlist[:5]
    selection = []
    for i in checked:
        if i.grade == currentgrade + 1:
            selection.append(i)
    if selection == []:
        return('No cards in top 5')
    selected = select_from(selection)
    player.handzone.add_card(selected)
    player.deckzone.remove_card(selected)
    shuffle(player.deckzone.cardlist)
    print('Select two cards to remove from the game:')
    print('First selection:')
    firstassist = select_from(player.handzone.cardlist)
    player.assistzone.add_card(firstassist)
    player.handzone.remove_card(firstassist)
    print('Second selection:')
    secondassist = select_from(player.handzone.cardlist)
    player.assistzone.add_card(secondassist)
    player.handzone.remove_card(secondassist)

def choose_guardians(player,restriction=None,nosentinel=False):
    guardlist = []
    for i in player.handzone.cardlist:
        if nosentinel and 'Sentinel' in i.tags:
            continue
        elif restriction:
            if i.grade in restriction:
                continue
        else:
            guardlist.append(i)
    while True:
        print('Choose a guardian from your hand?')
        print([x.name for x in guardlist])
        guardyesno = select_from_list('Yes','No')
        if guardyesno == 'Yes':
            guardcard = select_from(guardlist)
            player.guardzone.add_card(guardcard)
            player.handzone.remove_card(guardcard)
        else:
            break
    guardlist = []
    for j in list(filter(lambda x: (x.card != None), player.field)):
        if j.card.canintercept:
            guardlist.append(j)
    if guardlist != []:
        while True:
            print('Intercept?')
            print([x.card.name for x in guardlist])
            guardyesno = select_from_list('Yes','No')
            if guardyesno == 'Yes':
                guardcard = select_from([x.card.name for x in guardlist])
                player.guardzone.add_card(guardcard.card)
                player.guardcard.card=None
            else:
                break
    
def stand_phase(player):
    # check if skill prevents unit from standing during stand phase
    # skills that activate at start of stand phase
    # stand player's units
    unitfield = list(filter(lambda x: (x.card != None), player.field))
    for c in unitfield:
        c.card.isrest=False
    
def main_phase(player):
    ###option list
    # Call from Hand any cards with grade <= vanguard's
    # Activate Skills, call a "get potential skills that can activate right now" list function
    # Swap Columns (left or Right only)
    # Enter Battle Phase
    print('Main Phase. Please select from the following options:')
    while True:
        optionlist = ['Call from Hand','Activate Skill','Swap Columns','To Battle Phase']
        selection = select_from_list(optionlist)
        if selection == 'Call from Hand':
            print(player.centerfront.card.grade)
            print(player.handzone.cardlist[0].grade)
            calllist = []
            for i in player.handzone.cardlist:
                if i.grade <= player.centerfront.card.grade:
                    calllist.append(i)
            callselect = select_from(calllist)
            print('Which circle do you want to call to?')
            select_from(list(filter(lambda x: (x.name != 'centerfront'), player.field))).call_card(callselect)
        elif selection == 'Activate Skill':
            pass
        elif selection == 'Swap Columns':
            colselect = select_from_list(['Right','Left'])
            if colselect == 'Right':
                player.rightfront.card, player.rightback.card = player.rightback.card, player.rightfront.card
        else:
            return
    

def ride_phase(player):
    # generate list of rideable cards, ordered by Grade
    ridelist = []
    for i in player.handzone.cardlist:
        if i.grade == player.centerfront.card.grade + 1 or i.grade == player.centerfront.card.grade:
            ridelist.append(i)
    # If list doesn't contain card += Vangaurd's grade and that grade is less than 3:
    if ridelist == []:
        g_assist(player)
    # Now post-assist, check for rideability
    if ridelist != []:
        sorted(ridelist, key=lambda x: x.grade, reverse=True)
        print('Do you wish to ride one of these cards?')
        print([x.name for x in ridelist])
        rideyesno = select_from_list(['Yes','No'])
        if rideyesno == 'Yes':
            ridecard = select_from(ridelist)
            player.ride_card(ridecard)
            player.handzone.remove_card(ridecard)

def battle_phase(player,opponent):
    # Activate any skills on start of battle phase
    ###option list
    collist = []
    for i in player.field:
        if i.row == 1:
            collist.append(i)
    # Choose column, including Accel circles
    chosencol = select_from(collist)
    behind = list(filter(lambda x: (x.column == chosencol.column and x.row == 2), player.field))[0]
    if behind.card:
        if behind.card.boost and behind.card.isrest == False:
            print('Boost with {}?'.format(behind.card.name))
            if select_from_list(['Yes','No']) == 'Yes':
                behind.card.isrest = True
                chosencol.card.boostedpower += behind.card.current_power()
    # Select opponent's column, including accel circles
    oppcollist = []
    for i in opponent.field:
        if i.row == 1:
            collist.append(i)
    targetcol = select_from(oppcollist)
    # Activate any Skills on attack
    ### Attack!!!
    choose_guardians(opponent)
    # Calculate opponent's shield based on power + sum of shield on guard
    # Activate any When placed on G skills, including Sentinels
    # Drive Check if applicable
    # Skills upon Drive Check
    if attack(chosencol.card,targetcol.card):
        #damage check
        pass
    # Compare current power of attacker with opponent's target, if power >= target's power, inflict damage = attacker's critical

    # skills upon recieving damage
    # skills when unit is sent to dropzone from Guard circle
    # skills when unit is sent to dropzone from field (rear guards only)
    # skills when player's unit's attack hits or "after the battle that this unit attacked/boosted an attack at a vanguard"
    # Return to loop for next attack of units that are not tapped
    # Otherwise, only option left will be Procede to End Phase
    

def end_phase(player):
    # gather list of end of turn skills
    # including mandatory skills
    # choose skills from that list
    # once mandatory skills are done allow player to end turn
    # end turn, continuous skills end, boosted power returns to normal, boosted crit/drive/grade etc as well.
    pass

def attack(attacker,target):
    attacker.isrest = True
    print('{} attacks {}.  {} Power vs {} Power.'.format(attacker.name, target.name, attacker.current_power(), target.current_power()))
    if attacker.current_power() >= target.current_power():
        print('Attack hits, proceding to damage step')
        return True
    else:
        return False

def start_turn(player):
    global player1
    global player2
    global turnplayer
    global turncount
    if turncount == 0:
        mulligan(player)
        mulligan(get_opponent(player))
    turncount += 1
    #begin continuous skills for turn
    #start of turn skills

def turn(player):
    start_turn(player)
    stand_phase(player)
    ride_phase(player)
    main_phase(player)
    battle_phase(player,get_opponent(player))
    end_phase(player)

def game(p1,p2):
    global player1
    global player2
    global sentinel
    global turnplayer
    global turncount
    player1, player2 = p1, p2
    decide_first(player1,player2)
    turnplayer = player1
    player1.deck_init()
    player2.deck_init()
    player1.centerfront.call_card(player1.firstvan)
    player2.centerfront.call_card(player2.firstvan)
    draw(player1,5)
    draw(player2,5)
    while True:
        turn(turnplayer)
        change_turnplayer()
    
        

In [55]:
import pandas as pd


### Import the card list
cardlist = pd.read_csv('cardlist.csv',index_col=0)
carddb = {}
for x in cardlist.itertuples():
    carddb[x.Index] = Card(*x)

### Import the decklists

p1dek = {5: -1, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 2, 14: 4, 15: 4, 16: 3, 17: 4, 18: 4} #Royal Paladin Trial Deck
p2dek = {19: -1, 20: 4, 21: 4, 22: 4, 23: 4, 24: 4, 25: 4, 26: 4, 27: 2, 28: 4, 29: 4, 30: 3, 31: 4, 32: 4} #Kagero Trial Deck
#note for testing purposes, these two Trial Decks are included automatically

### Global vars

player1 = None
player2 = None
turnplayer = player1
sentinel = None
turncount = 0

In [56]:
Player1 = Player('RoyalPaladin',p1dek)
Player2 = Player('Kagero',p2dek)

game(Player1, Player2)

# print([x.name for x in Player1.handzone.cardlist])
# print([x.name for x in Player2.handzone.cardlist])

# draw(Player1, 1)

# print([x.name for x in Player1.handzone.cardlist])
# print([x.name for x in Player2.handzone.cardlist])

You are going Second!
Your hand is currently:
['Dragon Knight, Burj', 'Demonic Dragon Mage, Rakshasa', 'Demonic Dragon Mage, Rakshasa', 'Dragon Monk, Genjo', 'Dragon Knight, Nehalem']
How many cards would you like to mulligan?
0
Your hand is currently:
['Bringer of Good Luck, Epona', 'Knight Squire, Allen', 'Flogal', 'Sage of the Arts, Jauron', 'Sage of the Arts, Jauron']
How many cards would you like to mulligan?
0
Do you wish to ride one of these cards?
['Dragon Knight, Burj', 'Demonic Dragon Mage, Rakshasa', 'Demonic Dragon Mage, Rakshasa', 'Dragon Monk, Genjo']
Select from the following:
0 : Yes
1 : No
0
Select from the following:
0 : Dragon Knight, Burj
1 : Demonic Dragon Mage, Rakshasa
2 : Demonic Dragon Mage, Rakshasa
3 : Dragon Monk, Genjo
0
Main Phase. Please select from the following options:
Select from the following:
0 : Call from Hand
1 : Activate Skill
2 : Swap Columns
3 : To Battle Phase
0
1
0
Select from the following:
0 : Demonic Dragon Mage, Rakshasa
1 : Demonic Drago

ValueError: invalid literal for int() with base 10: '.'